In [ ]:
  !apt-get install poppler-utils


In [ ]:
!pip install pdf2image

In [ ]:
!pip install easyocr

In [ ]:
!pip install opencv-python

In [ ]:
!pip install xlsxwriter

In [ ]:
#O nltk ja está no collab, mas o nltk_contrib não 
!pip install nltk


In [ ]:
!pip install spacy

In [ ]:
from pdf2image import convert_from_path
import easyocr
import numpy as np
import PIL
from PIL import ImageDraw
import pandas as pd

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')

In [ ]:
import spacy

In [ ]:
import spacy

In [ ]:
import csv
try:
    from PIL import Image
except ImportError:
    import Image

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
import os
import string
from google.colab import drive

In [ ]:
from google.colab import drive

workdir_path = "/content/drive/My Drive/POC"

drive.mount('/content/drive')


os.chdir(workdir_path)

In [ ]:
reader = easyocr.Reader(['en','la'])


In [ ]:
pdf_invoice_name = 'Modelo Invoice Petrobras.pdf'
variant_name = 'a2_'

In [ ]:
invoices = convert_from_path(pdf_invoice_name)

In [ ]:
from IPython.display import display, Image

In [ ]:
display(invoices[0])

In [ ]:
#Salvando a imagem da invoice original
invoices[0].save(variant_name + "img_invoice.jpg")

In [ ]:
bounds = reader.readtext(np.array(invoices[0]),min_size=0,slope_ths=0.2,ycenter_ths=0.7,height_ths=0.6,width_ths=0.8,decoder='beamsearch',beamWidth=10)
bounds

In [ ]:
def draw_boxes(image, bounds, color='yellow',width=3,contador=0):
  draw = ImageDraw.Draw(image)
  for bound in bounds:
    p0, p1, p2, p3 = bound[0]
    draw.line([*p0, *p1, *p2, *p3, *p0], fill = color, width = width)  
  return image

draw_boxes(invoices[0],bounds,color='yellow')

In [ ]:
invoices[0].save(variant_name + "img_invoice.jpg")

In [ ]:
#Vamos criar uma imagem do zero e aplicar os textos sobre esta imagem 

from PIL import Image

img_only_texts = Image.new(invoices[0].mode, invoices[0].size,color = 'white')

img_only_texts

In [ ]:
# vamos o ler pdf de novo para garantir uma magem limpa da invoice sem as caixa amarelas geradas anteriormente, para termos apenas o texto.
invoices = convert_from_path(pdf_invoice_name)

In [ ]:

def copy_text_boxes(image,blank_image, bounds):
 for bound in bounds:
    p0, p1, p2, p3 = bound[0]
    region = image.crop((p0[0], p0[1], p2[0], p2[1] ))
    blank_image.paste(region,box= (p0[0], p0[1], p2[0], p2[1] ))
 return blank_image

copy_text_boxes(invoices[0],img_only_texts,bounds)

In [ ]:
img_only_texts.save(variant_name + "img_only_texts.jpg") 

In [ ]:
#importando a imagem da invoice somente com os textos para o formato da biblioteca opencv onde iremos trabalhar o uso de detecção morfológica

img = cv2.imread(variant_name + "img_only_texts.jpg")

In [ ]:

# limpa as cores
pre = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.figure(figsize = (500,50))
plotting = plt.imshow(pre)
plt.show()

In [ ]:

# Otsu threshold
pre = cv2.threshold(pre, 250, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# dilatar a imagem para transfromar os textos em um bloco sólido.
cpy = pre.copy()
morph_size=(6, 6)
struct = cv2.getStructuringElement(cv2.MORPH_RECT, morph_size)
cpy = cv2.dilate(~cpy, struct, anchor=(-1, -1), iterations=2)
#vamos erodir para tentar chegar a um imagem mais precisa
cpy = cv2.erode(~cpy, struct, anchor=(-1, -1), iterations=1)

pre = ~cpy

pre = 255-pre

plt.figure(figsize = (500,50))
plotting = plt.imshow(pre)
plt.show()



In [ ]:
#vasmo encontrar as caixas
min_text_height_limit=50
max_text_height_limit=600

contours, hierarchy = cv2.findContours(pre, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# criar as caixas  que cricrundam os textso baseados no seu tamanho
boxes = []
for contour in contours:
  box = cv2.boundingRect(contour)
  h = box[3]
  if min_text_height_limit < h < max_text_height_limit:
    boxes.append(box)
  
boxes

In [ ]:
# Criar um lista de outerboxes
def cria_lista_outerbox(box_list):
  p1 = [0,0]
  p2 = [0,0]
  p3 = [0,0]
  p4 = [0,0]
  outerbox = []

  for box in box_list:

    p1[0] = box[0]
    p1[1] = box[1]
    p2[0] = box[0]+box[2]
    p2[1] = box[1]
    p3[0] = box[0] + box[2]
    p3[1] = box[1] + box[3]
    p4[0] = box[0] 
    p4[1] = box[1] + box[3]

    outerbox.append([p1,p2,p3,p4])

    p1 = [0,0]
    p2 = [0,0]
    p3 = [0,0]
    p4 = [0,0]
  return(outerbox)

outerboxes = cria_lista_outerbox(boxes)

In [ ]:
vis_boxes = img_only_texts.copy()

def draw_outerboxes(image, bounds, color='yellow',width=3,contador=0):
  draw = ImageDraw.Draw(image)
  for bound in bounds:
    p0, p1, p2, p3 = bound
    draw.line([*p0, *p1, *p2, *p3, *p0], fill = color, width = width)  
  return image

draw_outerboxes(vis_boxes,outerboxes,color='blue')

In [ ]:
#vasmo sobrepor a imagem as caixas que demostram a estutura inferida pela análise morfológica com a estutura obtida de caixas de txto obtida do ocr
draw_boxes(vis_boxes,bounds,color='yellow')

In [ ]:
vis_boxes.save(variant_name + 'img_inv_outb_text.jpg')

In [ ]:
df_invoice = pd.DataFrame(data = bounds)
df_invoice.columns = ['bounds','text','width']
df_invoice["outerbox_number"] = np.nan
df_invoice["line"] = np.nan
df_invoice.head(10)

In [ ]:
outerboxes

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

def define_box_nmbr(innerbox):
  box_nmbr = -1
  q1,q2,q3,q4 = innerbox
  p1 = Point(q1)
  p2 = Point(q2)
  p3 = Point(q3)
  p4 = Point(q4)
  for idx,box in enumerate(outerboxes):
    polygon = Polygon(box)
    if polygon.contains(p1) == True or polygon.contains(p2) == True or polygon.contains(p3) == True or polygon.contains(p4) == True:
      box_nmbr = idx

  return box_nmbr

In [ ]:
df_invoice["outerbox_number"] = df_invoice['bounds'].apply(define_box_nmbr)
#df_invoice.head(50)
df_invoice[df_invoice["outerbox_number"]>0]

In [ ]:
# Agora vamos adcionar a linha

y_anterior = -1

linha = 0
for index, row in df_invoice.iterrows():
#  print(y_anterior,p1[1])
  p1,p2,p3,p4 = row['bounds']
  if p1[1] > y_anterior   and y_anterior != -1:
    y_anterior = p4[1]
    linha = linha + 1
    df_invoice.at[index,'line'] = int(linha)
#    print('depois ' + str(linha) + str(p1))
  elif y_anterior == -1:
    y_anterior = p4[1]
    linha = 1
    df_invoice.at[index,'line'] = int(linha)
#    print('primeiro ' + str(linha) + str(p1))
  else: 
    df_invoice.at[index,'line'] = int(linha)
    y_anterior = p4[1]
    


In [ ]:
df_invoice.dtypes

In [ ]:
#Corrigindo o tipo da linha
df_invoice["line"] = df_invoice['line'].astype('int')

In [ ]:
df_invoice.head(50)

In [ ]:
# Agora vamos começar a identificar os dados para extração de forma estruturada. Primimro passo será identifcar o grid de itens. 
# usaresmo três marcadores comuns a todas as invoices
# - Os itens de uma invoice precisam ter a quantidade discrimnada
# - Os itens de uma invoice precisam ter preço unitário
# - os itens precisam ter um preço total calculado por cada item
# Esses três pontos podem ser encontradso em qualquer invoice e morfologicamente o grid é uma estrutura um matriz de linhas por colunas em que
# as linhas de cima são só cabeçalhos de itens e as debaixo são as linhas de item. Faremos isso primeiro identificando o numero da linha de cabeçalho (line).
# de posse dessa linha de cabeçalho vamos obter as colunas que já foram agrupadas morfológicamente pelo itens outerbox como item que devem ser agrupados sob o mesmo rótulo

from difflib import SequenceMatcher

def ish1col(col_name):
  type_of_column = np.nan
  for qty_name in ['quantity','qty','quant','unit price','un price','unit pr','un pr','un px']:
    if SequenceMatcher(None, qty_name, col_name.lower()).ratio()  > 0.9:
      type_of_column = 'H1'
  return type_of_column

df_invoice['type'] = np.nan

df_invoice['type'] = df_invoice['text'].apply(ish1col)


In [ ]:
df_is_h1 = df_invoice[df_invoice['type'] == 'H1']
df_is_h1.head()

In [ ]:
#pegando o número da linha e verificando se ele é um só
df_invoice[df_invoice['type'] == 'H1']['line'].unique()

In [ ]:
# Ok é um só vamos pegar o numero da linha
header_grid_line = df_invoice[df_invoice['type'] == 'H1']['line'].unique()[0]
print(header_grid_line) 

In [ ]:
#vasmo a aplicar a marcação cabeçalho de coluna em todas a linhas do grid
def seth1(line):
  if line == header_grid_line:
    return 'H1'

df_invoice['type'] = df_invoice['line'].apply(seth1) 

In [ ]:
df_invoice[df_invoice['type'] == 'H1']

In [ ]:
#agora vamos marcar os itens pelo número da caixa
grid_outb_numbers = df_invoice[df_invoice['type'] == 'H1']['outerbox_number'].unique()
grid_outb_numbers

In [ ]:

def set_h2(row):
  if row[3] != -1:
    if row[3] in grid_outb_numbers and row[4] != header_grid_line:
      return 'H2'
    elif row[3] in grid_outb_numbers and row[4] == header_grid_line:
      return 'H1'
  else:
    if row[3] in grid_outb_numbers and row[4] == header_grid_line+1:
      return 'H2'
    elif row[3] in grid_outb_numbers and row[4] == header_grid_line:
      return 'H1'
    

def seth1_2(outbnumber):
  if outbnumber in grid_outb_numbers:
    return 'H1'
#df_invoice['type'] = df_invoice['outerbox_number'].apply(seth1_2) 
df_invoice['type'] = df_invoice.apply(set_h2,axis = 1) 


In [ ]:
df_invoice[df_invoice['type'] == 'H2']


In [ ]:
#Vamos vasmo buscar as outras informações que são dos nosso interesse, são elas:


# - Número da invoice.
# - Comprador do produto.
# - Consignatário (responsável pela guarda do produto até que ele seja entregue ao cliente)
# - FOB (termo comercial que se refere ao preço do produto sem acréscimo de frete e seguro)
# - Frete.
# - Seguro.
# - Valor total da invoice
# - Data de missão da invoice
# - Companhia emitente da invoice.

# Número da invoice, Comprador,Consignatário,Companhia emitente da invoice e Data de missão da invoice ficam sempre na priemria parte da invoice antes do grid
#FOB , Frete e Seguro ficam na segunda parte da invoice abaixo do grid 

# Vamos quebrar o data frame original em três partes para que possamos analisar as informações




In [ ]:
#splitanda a primera parte 
corte_1 = df_invoice[df_invoice['type'] == 'H1'].index[0] 
corte_1

In [ ]:
df_invoice1 = df_invoice.loc[0:corte_1]
df_invoice1.tail()

In [ ]:
len_grid = len(df_invoice[df_invoice['type'] == 'H1']) + len(df_invoice[df_invoice['type'] == 'H2'])
len_grid

In [ ]:
df_invoice2 = df_invoice.loc[corte_1+len_grid:]
df_invoice2.head()

In [ ]:
df_invoice_grid  = df_invoice.iloc[corte_1:corte_1+len_grid]
df_invoice_grid.tail(20)

In [ ]:
nbr_grid_columns = df_invoice_grid['type'].value_counts()
nbr_grid_columns

In [ ]:
# número de colunas do cabeçalho
nbr_gri_columns_header = nbr_grid_columns[nbr_grid_columns.keys() == 'H1'][0]
nbr_gri_columns_header

In [ ]:
# número de linhas do grid
nbr_grid_lines = len(df_invoice_grid['line'].unique())

In [ ]:
nbr_grid_lines

In [ ]:
# vamos criar um matriz multidimensional para gaurdar o grid mas antes vamos verificar se a opencv conseguiu agrupar as colunas do grid
if df_invoice_grid['outerbox_number'].unique()[0] == -1:
  flag_no_outerbox_cols = 'X'
else:
  flag_no_outerbox_cols = ''
flag_no_outerbox_cols

In [ ]:
grid_matrix = [['']*nbr_gri_columns_header for i in ['']*nbr_grid_lines]
grid_matrix

In [ ]:
#pegando as colunas do grid
arr_grid = df_invoice_grid['text'].to_numpy()
for i in range(nbr_gri_columns_header):
  grid_matrix[0][i] = arr_grid[i]

print(grid_matrix)
 

In [ ]:
nbr_gri_columns_header

In [ ]:
arr_i_grid =df_invoice_grid.to_numpy()
arr_i_grid = np.delete(arr_i_grid, np.s_[0:nbr_gri_columns_header], axis = 0)  
arr_i_grid = np.delete(arr_i_grid, 0, axis = 1)  
arr_i_grid = np.delete(arr_i_grid, 1, axis = 1)
arr_i_grid = np.delete(arr_i_grid, 3, axis = 1)
arr_i_grid

In [ ]:
if flag_no_outerbox_cols != 'X': # caso a opencv não tenha conseguido detectat as colunas do grid

  b = -1
  l = -1
  txt_anterior = ''
  delete_lines = []
  for idx,line in enumerate(arr_i_grid):
    if line[1] == b and l == line[2]:
  #    print(str(line[1]) + '- ' + str( line[2]))
  #    print(idx)
      line[0] = txt_anterior + ' ' + line[0]
      delete_lines.append(idx-1)
  #    arr_i_grid = np.delete(arr_i_grid, idx - 1, axis = 0) 
    b = line[1]
    l = line[2]
    txt_anterior = line[0]

  arr_i_grid

In [ ]:
if flag_no_outerbox_cols != 'X':
  delete_lines

In [ ]:
if flag_no_outerbox_cols != 'X':
  arr_i_grid = np.delete(arr_i_grid, delete_lines, axis = 0)
  arr_i_grid

In [ ]:
#pegando as linhas
if flag_no_outerbox_cols != 'X':
  arr_i_grid = np.delete(arr_i_grid, [1,2], axis = 1)

  arr_i_grid

In [ ]:
#pegando as colunas do grid
#arr_grid = df_invoice_grid['text'].to_numpy()
k = 0
arr_i_grid_list = arr_i_grid.tolist()
for i in range(1,nbr_grid_lines):  
  for j in range(nbr_gri_columns_header):
    grid_matrix[i][j] = arr_i_grid_list[k][0]
    k = k + 1

grid_matrix


In [ ]:
df_grid = pd.DataFrame(data = grid_matrix)
df_grid.columns = df_grid.iloc[0]
df_grid = df_grid[1:]
df_grid.head()


In [ ]:
#Agora utilizaremos o NLTK para obter as ocorrêcias dos valores que nos interessam nas partes de cabeçalho anterios e psoterios da invoice.
#Primeiro iremos concatenar os todos da prte da cima da invvoice em um unico grande texto dividiso por cada linhas. Entretanto, se houve um caixa de texto identificada pelo OCr
#vamos concatenar essa caixa como um texto em uma única linha
df_invoice1.head()



In [ ]:
inv_header_text = '. '.join(df_invoice1.groupby(['line','outerbox_number'])['text'].apply(' '.join).array)
print(inv_header_text)

In [ ]:
w_h_tokens = nltk.word_tokenize(inv_header_text)
w_h_tokens


In [ ]:
p_w_h_tokens = nltk.pos_tag(w_h_tokens)
p_w_h_tokens

In [ ]:
from difflib import SequenceMatcher

#Essa função será usada paras os caso simples de par chave e valor: são eles numero da invoice, FOB, Freight, FOB, Insurance, Fob+Taxes, Total Ammount

def find_values_text(p_w_h_tokens,invoice_field,pos_tag = 'CD'):

  invoice_candidate_values = []
  invoice_candidate_fields = []
  invoice_results = []
  for i, (word, pos) in enumerate(p_w_h_tokens):
  #    if(pos == 'NNP'):
      if SequenceMatcher(None, invoice_field, word.lower()).ratio()  > 0.9:
  #            print(str(i) + '-' + word)
           invoice_candidate_fields.append([i,word,pos])
      if (pos == pos_tag):
        invoice_candidate_values.append([i,word,pos])
  #      print(str(i) + '-' + word)
#  print(invoice_candidate_fields)
#  print(invoice_candidate_values)

  #vasmo pegar a referência e a invoice de menor distancia entre eles
  for fields in invoice_candidate_fields:
    f_idx,f_word,f_pos = fields
    for values in invoice_candidate_values:
      v_idx,v_word,v_pos = values
      distance = v_idx - f_idx 
      if distance > 0:
        invoice_results.append([f_idx,f_word,f_pos,v_idx,v_word,v_pos,distance])
  invoice_results = sorted(invoice_results,key=lambda x: x[6])
  return invoice_results

invoice_analisis = find_values_text(p_w_h_tokens,'invoice')
print(invoice_analisis)


In [ ]:
#Vamos começar a montar o dataframe
if len(invoice_analisis) > 0:
#ecnontrou candidatos? se sim vasmo criar pegar o primeiro resultado   
  invoice_name  = invoice_analisis[0][1]
  invoice_value = invoice_analisis[0][4]
  df_invoice_header =  pd.DataFrame(data = {invoice_name:[invoice_value]})
else:
  df_invoice_header =  pd.DataFrame(data = {'Invoice':['']})

df_invoice_header.head()


In [ ]:
invoice_analisis = find_values_text(p_w_h_tokens,'buyer','NNP')
print(invoice_analisis)

In [ ]:
if len(invoice_analisis) > 0:
#ecnontrou candidatos? se sim para o caso do buyer vamos pegar o oids primeiros nomes
  buyer_name  = invoice_analisis[0][1] 
  buyer_value = invoice_analisis[0][4] + ' ' + invoice_analisis[1][4]
  df_invoice_header[buyer_name] =  buyer_value
else:
  df_invoice_header['Buyer'] =  ''

df_invoice_header.head()

In [ ]:
invoice_analisis = find_values_text(p_w_h_tokens,'consignee','NNP')
print(invoice_analisis)

In [ ]:
if len(invoice_analisis) > 0:
#ecnontrou candidatos? se sim para o caso do consignee vamos pegar o oids primeiros nomes
  conignee_name  = invoice_analisis[0][1] 
  conignee_value = invoice_analisis[0][4] + ' ' + invoice_analisis[1][4]
  df_invoice_header[conignee_name] =  conignee_value
else:
  df_invoice_header['Consignee'] =  ''

df_invoice_header.head()

In [ ]:
inv_bottom_text = '. '.join(df_invoice2.groupby(['line','outerbox_number'])['text'].apply(' '.join).array)
print(inv_bottom_text)

In [ ]:
w_b_tokens = nltk.word_tokenize(inv_bottom_text)
w_b_tokens

In [ ]:
p_w_b_tokens = nltk.pos_tag(w_b_tokens)
p_w_b_tokens

In [ ]:
invoice_analisis = find_values_text(p_w_b_tokens,'fob')
print(invoice_analisis)

In [ ]:

if len(invoice_analisis) > 0:
#ecnontrou candidatos? se sim vasmo criar pegar o primeiro resultado   
  fob_name  = invoice_analisis[0][1]
  fob_value = invoice_analisis[0][4]
  df_invoice_header[fob_name] =  fob_value
else:
  df_invoice_header['FOB'] =  ''

df_invoice_header.head()

In [ ]:
invoice_analisis = find_values_text(p_w_b_tokens,'freight')
print(invoice_analisis)

In [ ]:
if len(invoice_analisis) > 0:
#ecnontrou candidatos? se sim vasmo criar pegar o primeiro resultado   
  freight_name  = invoice_analisis[0][1]
  freight_value = invoice_analisis[0][4]
  df_invoice_header[freight_name] =  freight_value
else:
  df_invoice_header['Freight'] =  ''

df_invoice_header.head()

In [ ]:
invoice_analisis = find_values_text(p_w_b_tokens,'insurance')
print(invoice_analisis)

In [ ]:
if len(invoice_analisis) > 0:
#ecnontrou candidatos? se sim vasmo criar pegar o primeiro resultado   
  insurance_name  = invoice_analisis[0][1]
  insurance_value = invoice_analisis[0][4]
  df_invoice_header[insurance_name] =  insurance_value
else:
  df_invoice_header['Insurance'] =  ''

df_invoice_header.head()

In [ ]:
invoice_analisis = find_values_text(p_w_b_tokens,'taxes')
print(invoice_analisis)

In [ ]:
invoice_analisis = find_values_text(p_w_b_tokens,'total')
print(invoice_analisis)

In [ ]:
if len(invoice_analisis) > 0:
#ecnontrou candidatos? se sim vasmo criar pegar o primeiro resultado   
  total_name  = invoice_analisis[0][1]
  total_value = invoice_analisis[0][4]
  df_invoice_header[total_name] =  total_value
else:
  df_invoice_header['Total'] =  ''

df_invoice_header.head()

In [ ]:
df_invoice_header.head()

In [ ]:
invoice_analisis = find_values_text(p_w_h_tokens,'date')
print(invoice_analisis)

In [ ]:
# Vasmo utilizar a biblioteca spacy para obter a data da invoice (data de emissão )
nlp = spacy.load("en")

doc_header_invoice = nlp(inv_header_text)
header_entities =  [(i,i.label_, i.label) for i in doc_header_invoice.ents]
header_entities

In [ ]:
#Conforme observação da maioria das invoices a data ocorre logo no início sendo assim vasmo pegar a primeira data que aparece na invoice

for index, entity in enumerate(header_entities):
  if entity[1] == 'DATE':
    invoice_date = str(entity[0])

print(invoice_date)

In [ ]:
#encontrou candidatos? se sim vamos pegar o primeiro resultado   
df_invoice_header["Invoice Date"] = invoice_date

df_invoice_header.head()

In [ ]:
#para o emitente da invocie utilizaremos uma outra regra:
#- Vasmo concatenar as três primeiras linhas e vamos procurar os trigramas que terminam com um das seguintes abreviações que indicam empresa: ltd, gmbh, bv, sa, , inc

df_invoice_3lines =  df_invoice1[df_invoice1['line'] <= 3]
text_3_lines  = '. '.join(df_invoice_3lines.groupby(['line','outerbox_number'])['text'].apply(' '.join).array)
print(text_3_lines) 


In [ ]:
w_3_tokens = nltk.word_tokenize(text_3_lines)
#p_3_tokens = nltk.pos_tag(w_3_tokens)
w_3_tokens


In [ ]:
 company_trigrams =  nltk.trigrams(w_3_tokens)
 company_suffixes = ['ltd', 'gmbh', 'bv', 'sa','inc','company','b.v.']
 company_candidates = []

 for trigrama in company_trigrams:
    if trigrama[2].lower() in company_suffixes:
      company_candidates.append(trigrama[0] + ' ' + trigrama[1] + ' ' + trigrama[2])
    print(trigrama)
company_candidates

In [ ]:
if len(company_candidates) > 0:
#ecnontrou candidatos? se sim vamos pegar o primeiro resultado   
  company_name  = 'Company'
  company_value = company_candidates[0]
  df_invoice_header[company_name] =  company_value
else:
  df_invoice_header[company_name] =  ''

df_invoice_header.head()

In [ ]:
drive.mount('/content/drive')
os.chdir(workdir_path)
df_invoice_header.to_csv(variant_name + 'header.csv')
df_grid.to_csv(variant_name + 'grid.csv')